Combining Weather datasets & fire data

In [ ]:
-- Create a new table to hold the combined data
CREATE TABLE combined_weather_data (
    weather_date Text,
    weather_tavg Text,
    weather_tmin FLOAT,
    weather_tmax FLOAT,
    weather_prcp Text,
    weather_snow Text,
    weather_wdir Text,
    weather_wspd Text,
    weather_wpgt Text,
    weather_pres Text,
    weather_tsun Text,
    station_id Text,
    weather_station_name TEXT
);

INSERT INTO combined_weather_data
SELECT * FROM "weather_1995-1996";

-- Insert data from the second dataset (including 1996-01-01)
INSERT INTO combined_weather_data
SELECT * FROM "weather_1996-2005";

INSERT INTO combined_weather_data
SELECT * FROM "weather_2005-2013";

INSERT INTO combined_weather_data
SELECT * FROM "weather_2014-2015";

---count of rows in weather_1995-1996
SELECT count (*)
FROM "weather_1995-1996";

SELECT count(control_date)
FROM fires_data
WHERE control_date 

SELECT count(*)
FROM combined_weather_data;

-- Create a copy of a table in the new database

CREATE TABLE fires_data_V1 AS
SELECT * FROM fires_data;

-- Drop Unused Columns 

ALTER TABLE fires_data_v1 
DROP COLUMN discovery_doy;

ALTER TABLE fires_data_v1 
DROP COLUMN control_doy;

-- Update the table to add a new combined timestamp column
ALTER TABLE fires_data_v1 
ADD combined_control_date TIMESTAMP;

ALTER TABLE fires_data_v1 
ADD combined_discovery_date TIMESTAMP;

-- Update the new column with the combined timestamp
UPDATE fires_data_v1 
SET combined_control_date = CAST(control_date AS TIMESTAMP) + 
                            CAST(COALESCE(control_time, '00:00:00') AS INTERVAL HOUR TO SECOND);
                          
-- Update the new column with the combined timestamp
UPDATE fires_data_v1 
SET combined_discovery_date = CAST(discovery_date AS TIMESTAMP) + 
                              CAST(COALESCE(discovery_time, '00:00:00') AS INTERVAL HOUR TO SECOND);

SELECT Count(DISTINCT combined_control_date)
FROM fires_data_v1;

SELECT Count(DISTINCT combined_discovery_date)
FROM fires_data_v1;

--- CHECK how much fires got eather an discovery OR CONTROL date

SELECT count(DISTINCT combined_control_date) + count(DISTINCT combined_discovery_date)
FROM fires_data_v1;



SELECT count(*)
FROM fires_data_v1
WHERE combined_discovery_date ISNULL;


-- Drop the original date and time columns (optional)

ALTER TABLE fires_data_v1 
DROP COLUMN discovery_date,
DROP COLUMN discovery_time,
DROP COLUMN control_time,
DROP COLUMN control_date;

ALTER TABLE fires_data_v1_rdy_to_clean_rows 
DROP COLUMN fire_year;

---Adding NEW COLUMN WITH the difference IN time IF time IS known


ALTER TABLE fires_data_v1
ADD burning_time INTEGER; -- You can adjust the data type as needed


UPDATE fires_data_v1 
SET burning_time = 
    CASE
        WHEN combined_control_date IS NOT NULL AND combined_discovery_date IS NOT NULL THEN
            EXTRACT(DAY FROM combined_control_date - combined_discovery_date)
        ELSE
            NULL
    END;
   
SELECT count(burning_time)   
FROM fires_data_v1;

DELETE FROM fires_data_v1 
WHERE combined_discovery_date IS NOT NULL
      AND combined_discovery_date < '1995-01-01';

SELECT combined_discovery_date, unique_id, fire_size
FROM fires_data_v1_rdy_to_clean_rows 
ORDER BY combined_discovery_date asc;

--- Rename Fire_name & adapt null values to undefind 

UPDATE fires_data_v1_rdy_to_clean_rows 
SET fire_name = COALESCE(fire_name, 'undefined');

SELECT count(fire_name)
FROM fires_data_v1_rdy_to_clean_rows
WHERE fire_name = 'undefined';

UPDATE fires_data_v1_rdy_to_clean_rows 
SET gov_firecode = COALESCE(gov_firecode, 'undefined');

UPDATE fires_data_v1_rdy_to_clean_rows 
SET county = COALESCE(county, 'undefined');

SELECT count(DISTINCT county)
FROM fires_data_v1_rdy_to_clean_rows;

SELECT count(*)
FROM fires_data_v1_rdy_to_clean_rows 
WHERE burning_time IS NULL;

----Null values 816634

SELECT count(*)
FROM fires_data_v1_rdy_to_clean_rows
WHERE burning_time >= 360;

---108 fires burning longer then 1 year / we drop them to not get our avg adulterated over years

DELETE FROM fires_data_v1_rdy_to_clean_rows  
WHERE burning_time  >= 360;

--- check if it worked

SELECT count(*)
FROM fires_data_v1_rdy_to_clean_rows
WHERE burning_time >= 360;

--- 0
----------
--- CREATE a NEW COLUMN FOR state 

ALTER TABLE fires_data_v1_rdy_to_clean_rows 
ADD COLUMN state_name varchar;
------------

SELECT sum(fire_size_class = 'A'), sum(fire_size)
FROM fires_data_v1_rdy_to_clean_rows 
WHERE combined_discovery_date >= '2000-01-01'

---

ALTER TABLE fires_data_v1_rdy_to_clean_rows 
DROP COLUMN state;

-- Assuming you have a table called wild_fire_data with multiple columns
-- Concatenate all columns into a single string using CONCAT or || operator

SELECT md5(unique_id || state_name || source_reporting_unit_name || shape_hex || longitude || latitude || gov_firecode || fire_size_class || fire_size || fire_name || county || combined_discovery_date || combined_control_date || cause_descr || cause_code || burning_time) as hash_value
FROM fires_data_v1_rdy_to_clean_rows
GROUP BY hash_value
HAVING COUNT(*) > 1;

--- [Null]